In [1]:
from cassie import CassieEnv
import numpy as np

import time

env = CassieEnv("walking")

qpos = np.copy(env.sim.qpos())

jpos = qpos[env.pos_idx]

fbpos = qpos[0:7]



In [2]:
# Joint limits converted to radians from XML file

ROLL_LOW   = -0.26
ROLL_HIGH  =  0.39

YAW_LOW    = -0.39
YAW_HIGH   =  0.39

PITCH_LOW  = -0.87
PITCH_HIGH =  1.4

KNEE_LOW   = -2.86
KNEE_HIGH  = -0.64

FOOT_LOW   = -2.44
FOOT_HIGH  = -0.52

# achilles = .75 + pitch = 1
# not currently correct though
ACHILLES_LIMIT = 1.75

DELTA      = 0.05 # resolution for IK data set

while jpos[2] > PITCH_LOW and jpos[3] > KNEE_LOW:
#     qpos = np.copy(env.sim.qpos())
#     if qpos[10] < ACHILLES_ROD_LOW:
#         print(qpos[2])
    
    jpos[2] -= DELTA
    jpos[7] -= DELTA
    
    jpos[3] -= DELTA
    jpos[8] -= DELTA
    
    env.set_joint_pos(jpos, fbpos)
    #env.render()
    
# jpos[2] = PITCH_LOW
# jpos[3] = KNEE_LOW

# env.set_joint_pos(jpos, fbpos)
# env.render()

# input("")


pitch = PITCH_LOW
while pitch < PITCH_HIGH:
    pitch += DELTA
    
    knee = KNEE_LOW
    while knee < KNEE_HIGH:
            knee += DELTA
            
            jpos[2] = pitch
            jpos[7] = pitch

            jpos[3] = knee
            jpos[8] = knee

            env.set_joint_pos(jpos, fbpos, iters=1000)
            
            qpos = np.copy(env.sim.qpos())

            #if qpos[10] + qpos[2] < ACHILLES_LIMIT:
            #    continue 
                
            env.render()


    # ramp knee angle back down
    while jpos[3] > KNEE_LOW:
        jpos[3] -= 0.1
        jpos[8] -= 0.1
        
        env.set_joint_pos(jpos, fbpos, iters=1000)
        env.render()
    
                
# while i > -2.44:# change knee angle
#     jpos[4] = i
#     jpos[9] = i

#     i -= .1
    
#     fbpos = qpos[0:7]

#     env.set_joint_pos(jpos, fbpos)
        
#     env.render()
    
#     print(jpos[3])
    
#     time.sleep(1/100)

### 